In [1]:
# 모듈 가져오기
import pandas as pd
import statsmodels.api as sm
import itertools
import time

In [2]:
# 데이터 가져오기 - 도요타 중고차 가격 데이터
Toyota = pd.read_csv('./Data/ToyotaCorolla.csv')

In [3]:
# 데이터 확인
Toyota.head()

,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,1,0,0,0,1,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,1,0,1,0,0,0,1,0,0,0
2,3,?TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90,1,...,0,0,1,0,0,0,1,0,0,0
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90,0,...,0,0,1,0,0,0,1,0,0,0
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90,0,...,1,1,1,0,1,0,1,0,0,0


In [4]:
# 데이터 컬럼정보 확인
Toyota.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1436 entries, 0 to 1435
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                1436 non-null   int64 
 1   Model             1436 non-null   object
 2   Price             1436 non-null   int64 
 3   Age_08_04         1436 non-null   int64 
 4   Mfg_Month         1436 non-null   int64 
 5   Mfg_Year          1436 non-null   int64 
 6   KM                1436 non-null   int64 
 7   Fuel_Type         1436 non-null   object
 8   HP                1436 non-null   int64 
 9   Met_Color         1436 non-null   int64 
 10  Automatic         1436 non-null   int64 
 11  cc                1436 non-null   int64 
 12  Doors             1436 non-null   int64 
 13  Cylinders         1436 non-null   int64 
 14  Gears             1436 non-null   int64 
 15  Quarterly_Tax     1436 non-null   int64 
 16  Weight            1436 non-null   int64 
 17  Mfr_Guarantee 

In [5]:
# 데이터 확인
dummies = pd.get_dummies(Toyota['Fuel_Type'])

In [6]:
# 불필요한 변수 제거 및 더미변수 추가
remove_cols = ['Id', 'Model', 'Fuel_Type']
tmp = Toyota.drop(remove_cols, axis = 1)
toyota_new = pd.concat([tmp, dummies], axis = 1)

In [7]:
# 데이터 확인
toyota_new

,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,...,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar,CNG,Diesel,Petrol
0,13500,23,10,2002,46986,90,1,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
1,13750,23,10,2002,72937,90,1,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
2,13950,24,9,2002,41711,90,1,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
3,14950,26,7,2002,48000,90,0,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
4,13750,30,3,2002,38500,90,0,0,2000,3,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,7500,69,12,1998,20544,86,1,0,1300,3,...,0,1,1,1,0,0,0,0,0,1
1432,10845,72,9,1998,19000,86,0,0,1300,3,...,0,0,1,1,0,0,0,0,0,1
1433,8500,71,10,1998,17016,86,0,0,1300,3,...,0,0,0,1,0,0,0,0,0,1
1434,7250,70,11,1998,16916,86,1,0,1300,3,...,0,0,0,1,0,0,0,0,0,1


In [10]:
# 데이터 분할
toyota_added = sm.add_constant(toyota_new, has_constant = 'add')
X = toyota_added.drop('Price', axis = 1)
y = toyota_added['Price']

## AIC가가 가장 작은값을 가지는 모델 -> 최적 회귀모델

In [25]:
def processSubset(X, y, feature_set):
    model = sm.OLS(y, X[feature_set])
    regression = model.fit()
    AIC = regression.aic    
    return {'model' : regression, "AIC" : AIC}

def getBest(X,y,k):
    tic = time.time()
    results = list()
    for combo in itertools.combinations(X.columns.difference(['const']), k):
        combo = list(combo) + ['const']
        results.append(processSubset(X,y,feature_set = combo))
    models = pd.DataFrame(results)
    bestModel = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print('Processed', models.shape[0], 'models on', k, 'predictors in', (toc-tic), 'seconds')
    return models, bestModel

In [27]:
# 최적 모델 확인
best_model = getBest(X,y,len(X.columns.difference(['const'])) - 1)[1]

Processed 36 models on 35 predictors in 0.2154226303100586 seconds


In [28]:
# 최적 모델 summary
best_model['model'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.909
Model:                            OLS   Adj. R-squared:                  0.907
Method:                 Least Squares   F-statistic:                     436.4
Date:                Tue, 10 Nov 2020   Prob (F-statistic):               0.00
Time:                        22:14:43   Log-Likelihood:                -12088.
No. Observations:                1436   AIC:                         2.424e+04
Df Residuals:                    1403   BIC:                         2.442e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
ABS               -268.4855    112.874     -2.379      0.018    -489.905     -47.066
Age_08_04         -118.8924      3.452    -34.439      0.000    -125.665    -112.120
Airbag_1           118.3702    220.070      0.538      0.591    -313.331     550.071
Airbag_2           -77.3409    115.119     -0.672      0.502    -303.164     148.482
Airco              198.4050     79.304      2.502      0.012      42.838     353.972
Automatic          346.0122    134.701      2.569      0.010      81.776     610.249
Automatic_airco   2441.3807    167.488     14.576      0.000    2112.828    2769.934
BOVAG_Guarantee    491.4573    112.028      4.387      0.000     271.697     711.218
Backseat_Divider  -264.8703    114.059     -2.322      0.020    -488.614     -41.127
Boardcomputer     -267.0286    104.143     -2.564      0.010    -471.321     -62.736
CD_Player          208.7346     87.059      2.398      0.017      37.955     379.514
CNG              -1027.0632    198.165     -5.183      0.000   -1415.795    -638.332
Central_Lock       -88.7869    126.866     -0.700      0.484    -337.655     160.081
Cylinders           -0.0341      0.002    -17.406      0.000      -0.038      -0.030
Diesel             -68.0183    163.294     -0.417      0.677    -388.346     252.309
Doors               37.7441     35.647      1.059      0.290     -32.182     107.671
Gears              153.7564    174.886      0.879      0.379    -189.311     496.823
Guarantee_Period    66.0168     12.059      5.475      0.000      42.362      89.672
HP                  19.1014      3.121      6.121      0.000      12.980      25.223
KM                  -0.0164      0.001    -14.547      0.000      -0.019      -0.014
Metallic_Rim       205.9538     83.985      2.452      0.014      41.204     370.703
Mfg_Month          -95.4753      9.061    -10.537      0.000    -113.250     -77.701
Mfg_Year             0.7845      0.739      1.062      0.288      -0.664       2.233
Mfr_Guarantee      225.6117     64.495      3.498      0.000      99.094     352.129
Mistlamps          -50.1405     96.316     -0.521      0.603    -239.080     138.799
Petrol            1095.0730    168.351      6.505      0.000     764.827    1425.319
Power_Steering     -43.8298    249.214     -0.176      0.860    -532.702     445.042
Powered_Windows    418.8037    126.947      3.299      0.001     169.777     667.830
Quarterly_Tax       14.4207      1.621      8.898      0.000      11.242      17.600
Radio              538.3413    653.393      0.824      0.410    -743.390    1820.073
Radio_cassette    -639.7903    653.617     -0.979      0.328   -1921.961     642.381
Sport_Model        304.5579     77.978      3.906      0.000     151.591     457.524
Tow_Bar           -203.7666     69.185     -2.945      0.003    -339.485     -68.049
Weight              10.

### 전진선택법

In [34]:
# 변수 사전 정의
predictors = X.columns

# 전진 선택법
def forward(X,y,predictors):
    remainingPredictors = [p for p in X.columns.difference(['const']) if p not in predictors]
    tic = time.time()
    results = list()
    for p in remainingPredictors:
        results.append(processSubset(X, y, feature_set=predictors + [p] + ['const']))
        models = pd.DataFrame(results)
        bestModel = models.loc[models['AIC'].argmin(), :]
        toc = time.time()
        print('Processed', models.shape[0], 'models on', len(predictors) + 1, 'predictors in', (toc-tic))
        print('Selected predictors:', bestModel['model'].model.exog_names, 'AIC:' , bestModel['AIC'])
        return bestModel
    
# 전진 선택법 모델
def forward_model(X,y):
    fModels = pd.DataFrame(columns = ['AIC', 'model'])
    tic = time.time()
    predictors = list()
    for i in range(1, len(X.columns.difference(['const']))+1):
        forwardResult = forward(X,y,predictors)
        if i > 1:
            if forwardResult['AIC'] > fmodelBefore:
                break
        fModels.loc[i] = forwardResult
        predictors = fModels.loc[i]['model'].model.exog_names
        fmodelBefore = fModels.loc[i]['AIC']
        predictors = [k for k in predictors if k != 'const']
    toc = time.time()
    print("Total elapesed time : ", (toc - tic), "seconds.")
    return (fModels['model'][len(fModels['model'])])

In [36]:
# 최적회귀모델
forward_model(X,y).summary()

Processed 1 models on 1 predictors in 0.01094961166381836
Selected predictors: ['ABS', 'const'] AIC: 27476.22486484318
Processed 1 models on 2 predictors in 0.006992340087890625
Selected predictors: ['ABS', 'Age_08_04', 'const'] AIC: 25495.50966652165
Processed 1 models on 3 predictors in 0.004986286163330078
Selected predictors: ['ABS', 'Age_08_04', 'Airbag_1', 'const'] AIC: 25495.551241670393
Total elapesed time :  0.04687333106994629 seconds.


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.772
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     2428.
Date:                Tue, 10 Nov 2020   Prob (F-statistic):               0.00
Time:                        22:22:23   Log-Likelihood:                -12745.
No. Observations:                1436   AIC:                         2.550e+04
Df Residuals:                    1433   BIC:                         2.551e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
ABS         -625.9303    128.834     -4.858      0.000    -878.654    -373.206
Age_08_04   -176.3490      2.700    -65.323      0.000    -181.645    -171.053
const       2.111e+04    221.252     95.394      0.000    2.07e+04    2.15e+04
==============================================================================
Omnibus:                      348.274   Durbin-Watson:                   1.261
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2670.318
Skew:                           0.913   Prob(JB):                         0.00
Kurtosis:                       9.426   Cond. No.                         315.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 후진제거법

In [37]:
# 후진 제거법
def backward(X,y,predictors):
    tic = time.time()
    results = list()
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(X,y, list(combo) + ['const']))
    models = pd.DataFrame(results)
    bestModel = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print("Processed",models.shape[0],"models on",len(predictors)-1,
          "predictors in",(toc - tic))
    print("Selected predictors :",bestModel['model'].model.exog_names,
          ' AIC:',bestModel['AIC'])
    return bestModel
# 후진 제거법 모델
def backward_model(X,y):
    BModels = pd.DataFrame(columns = ['AIC', 'model'])
    tic = time.time()
    predictors = X.columns.difference(['const'])
    BmodelBefore = processSubset(X,y,predictors)['AIC']
    while (len(predictors)>1):
        backwardResult = backward(X,y,predictors)
        if backwardResult['AIC'] > BmodelBefore:
            break
        BModels.loc[len(predictors)-1] = backwardResult
        predictors = BModels.loc[len(predictors)-1]['model'].model.exog_names
        BmodelBefore = backwardResult['AIC']
        predictors = [k for k in predictors if k != 'const']
    
    
    toc = time.time()
    print("Total elapsed time :",(toc - tic), "seconds.")
    return BModels['model'].dropna().iloc[-1]

In [38]:
backward_model(X,y).summary()

Processed 36 models on 35 predictors in 0.23839163780212402
Selected predictors : ['ABS', 'Age_08_04', 'Airbag_1', 'Airbag_2', 'Airco', 'Automatic', 'Automatic_airco', 'BOVAG_Guarantee', 'Backseat_Divider', 'Boardcomputer', 'CD_Player', 'CNG', 'Central_Lock', 'Cylinders', 'Diesel', 'Doors', 'Gears', 'Guarantee_Period', 'HP', 'KM', 'Metallic_Rim', 'Mfg_Month', 'Mfg_Year', 'Mfr_Guarantee', 'Mistlamps', 'Petrol', 'Power_Steering', 'Powered_Windows', 'Quarterly_Tax', 'Radio', 'Radio_cassette', 'Sport_Model', 'Tow_Bar', 'Weight', 'cc', 'const']  AIC: 24242.24759492178
Processed 35 models on 34 predictors in 0.23041749000549316
Selected predictors : ['ABS', 'Age_08_04', 'Airbag_1', 'Airbag_2', 'Airco', 'Automatic', 'Automatic_airco', 'BOVAG_Guarantee', 'Backseat_Divider', 'Boardcomputer', 'CD_Player', 'CNG', 'Central_Lock', 'Cylinders', 'Diesel', 'Doors', 'Gears', 'Guarantee_Period', 'HP', 'KM', 'Metallic_Rim', 'Mfg_Month', 'Mfg_Year', 'Mfr_Guarantee', 'Mistlamps', 'Petrol', 'Powered_Windows

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.908
Model:                            OLS   Adj. R-squared:                  0.907
Method:                 Least Squares   F-statistic:                     608.7
Date:                Tue, 10 Nov 2020   Prob (F-statistic):               0.00
Time:                        22:25:51   Log-Likelihood:                -12091.
No. Observations:                1436   AIC:                         2.423e+04
Df Residuals:                    1412   BIC:                         2.436e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
ABS               -297.4536     87.734     -3.390      0.001    -469.557    -125.350
Age_08_04          -22.9327      9.007     -2.546      0.011     -40.601      -5.265
Airco              191.3195     75.749      2.526      0.012      42.726     339.913
Automatic          315.9247    132.939      2.376      0.018      55.146     576.704
Automatic_airco   2418.4800    157.138     15.391      0.000    2110.230    2726.730
BOVAG_Guarantee    497.0693    110.651      4.492      0.000     280.012     714.127
Backseat_Divider  -290.6908    103.011     -2.822      0.005    -492.762     -88.619
Boardcomputer     -269.4259    102.935     -2.617      0.009    -471.349     -67.503
CD_Player          219.7123     85.235      2.578      0.010      52.511     386.914
CNG              -4.523e+04   4166.923    -10.855      0.000   -5.34e+04   -3.71e+04
Cylinders        -5.303e+05   4.99e+04    -10.618      0.000   -6.28e+05   -4.32e+05
Diesel           -4.427e+04   4155.938    -10.653      0.000   -5.24e+04   -3.61e+04
Guarantee_Period    66.4281     11.864      5.599      0.000      43.156      89.700
HP                  19.0061      3.023      6.287      0.000      13.076      24.936
KM                  -0.0163      0.001    -14.599      0.000      -0.018      -0.014
Metallic_Rim       198.6701     78.694      2.525      0.012      44.300     353.041
Mfg_Year          1147.2139    108.044     10.618      0.000     935.271    1359.157
Mfr_Guarantee      225.5253     64.222      3.512      0.000      99.544     351.507
Petrol           -4.306e+04   4172.965    -10.320      0.000   -5.12e+04   -3.49e+04
Powered_Windows    342.3198     73.550      4.654      0.000     198.040     486.600
Quarterly_Tax       14.4267      1.575      9.159      0.000      11.337      17.517
Sport_Model        307.4723     73.520      4.182      0.000     163.253     451.692
Tow_Bar           -202.5525     67.671     -2.993      0.003    -335.300     -69.805
Weight              11.3668      1.047     10.857      0.000       9.313      13.421
cc                  -0.1116      0.077     -1.450      0.147      -0.263       0.039
const            -1.326e+05   1.25e+04    -10.618      0.000   -1.57e+05   -1.08e+05
==============================================================================
Omnibus:                      151.108   Durbin-Watson:                   1.714
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1183.057
Skew:                           0.051   Prob(JB):                    1.27e-257
Kurtosis:                       7.445   Cond. No.                     1.33e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.99e-20. 

### 단계적 방법

In [42]:
def Stepwise_model(X,y):
    stepModels = pd.DataFrame(columns = ['AIC', 'model'])
    tic = time.time()
    predictors = list()
    SmodelBefore = processSubset(X,y,predictors + ['const'])['AIC']
    for i in range(1, len(X.columns.difference(['const']))+1):
        forwardResult = forward(X,y,predictors)
        print('forward')
        stepModels.loc[i] = forwardResult
        predictors = stepModels.loc[i]['model'].model.exog_names
        predictors = [k for k in predictors if k != 'const']
        backwardResult = backward(X,y,predictors)
        if backwardResult['AIC'] < forwardResult['AIC']:
            stepModels.loc[i] = backwardResult
            predictors = stepModels.loc[i]['model'].model.exog_names
            smodelBefore = stepModels.loc[i]['AIC']
            predictors = [k for k in predictors if k != 'const']
            print('backward')
        if stepModels.loc[i]['AIC'] > SmodelBefore:
            break
        else:
            smodelBefore = stepModels.loc[i]['AIC']
    toc = time.time()
    print("Total elapsed time : ", (toc - tic), "seconds")
    return stepModels['model'][len(stepModels['model'])]

In [43]:
Stepwise_model(X, y).summary()

Processed 1 models on 1 predictors in 0.005980968475341797
Selected predictors: ['ABS', 'const'] AIC: 27476.22486484318
forward
Processed 1 models on 0 predictors in 0.004976749420166016
Selected predictors : ['const']  AIC: 27615.537867847517
Processed 1 models on 2 predictors in 0.008977651596069336
Selected predictors: ['ABS', 'Age_08_04', 'const'] AIC: 25495.50966652165
forward
Processed 2 models on 1 predictors in 0.00897669792175293
Selected predictors : ['Age_08_04', 'const']  AIC: 25516.97057066566
Processed 1 models on 3 predictors in 0.005984067916870117
Selected predictors: ['ABS', 'Age_08_04', 'Airbag_1', 'const'] AIC: 25495.551241670393
forward
Processed 3 models on 2 predictors in 0.008976936340332031
Selected predictors : ['ABS', 'Age_08_04', 'const']  AIC: 25495.50966652165
backward
Processed 1 models on 3 predictors in 0.00498652458190918
Selected predictors: ['ABS', 'Age_08_04', 'Airbag_1', 'const'] AIC: 25495.551241670393
forward
Processed 3 models on 2 predictors in

Processed 1 models on 3 predictors in 0.0029916763305664062
Selected predictors: ['ABS', 'Age_08_04', 'Airbag_1', 'const'] AIC: 25495.551241670393
forward
Processed 3 models on 2 predictors in 0.010970115661621094
Selected predictors : ['ABS', 'Age_08_04', 'const']  AIC: 25495.50966652165
backward
Processed 1 models on 3 predictors in 0.0049896240234375
Selected predictors: ['ABS', 'Age_08_04', 'Airbag_1', 'const'] AIC: 25495.551241670393
forward
Processed 3 models on 2 predictors in 0.007980823516845703
Selected predictors : ['ABS', 'Age_08_04', 'const']  AIC: 25495.50966652165
backward
Processed 1 models on 3 predictors in 0.003989458084106445
Selected predictors: ['ABS', 'Age_08_04', 'Airbag_1', 'const'] AIC: 25495.551241670393
forward
Processed 3 models on 2 predictors in 0.012965202331542969
Selected predictors : ['ABS', 'Age_08_04', 'const']  AIC: 25495.50966652165
backward
Processed 1 models on 3 predictors in 0.003989696502685547
Selected predictors: ['ABS', 'Age_08_04', 'Airba

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.772
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     2428.
Date:                Tue, 10 Nov 2020   Prob (F-statistic):               0.00
Time:                        22:33:00   Log-Likelihood:                -12745.
No. Observations:                1436   AIC:                         2.550e+04
Df Residuals:                    1433   BIC:                         2.551e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
ABS         -625.9303    128.834     -4.858      0.000    -878.654    -373.206
Age_08_04   -176.3490      2.700    -65.323      0.000    -181.645    -171.053
const       2.111e+04    221.252     95.394      0.000    2.07e+04    2.15e+04
==============================================================================
Omnibus:                      348.274   Durbin-Watson:                   1.261
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2670.318
Skew:                           0.913   Prob(JB):                         0.00
Kurtosis:                       9.426   Cond. No.                         315.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""